In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import xarray as xr
import struct
as_int = lambda f: struct.unpack('>l', struct.pack('>f', f))

### Making a list of all the file names

In [2]:
# Code to loop through every filename (working)
beginning = "3B42_Daily."
end = ".7.nc4"

def YearLoop(yb, ye):
    dates = []
    for i in np.arange(yb, ye+1):
        year = i
        for j in np.arange (1, 12+1):
            month = j
            if len(str(month)) < 2:
                month = str(0) + str(month)
            if j==1 or j==3 or j==5 or j==7 or j==8 or j==10 or (j==12 and year != 2019):
                for k in np.arange (1, 31+1):
                    day = k
                    if len(str(day)) < 2:
                        day = str(0) + str(day)
                    dates.append((str(year) + str(month) + str(day)))
            if j==4 or j==6 or j==9 or j==11 or (j == 12 and year == 2019):
                for k in np.arange (1, 30+1):
                    day = k
                    if len(str(day)) < 2:
                        day = str(0) + str(day)
                    dates.append((str(year) + str(month) + str(day)))
            if j == 2 and year%4 == 0:
                for k in range (1, 29+1):
                    day = k
                    if len(str(day)) < 2:
                        day = str(0) + str(day)
                    dates.append((str(year) + str(month) + str(day)))
            if j == 2 and year%4 != 0:
                for k in range (1, 28+1):
                    day = k
                    if len(str(day)) < 2:
                        day = str(0) + str(day)
                    dates.append((str(year) + str(month) + str(day)))
    return dates

middle = YearLoop(1998, 1998)[0]
filename = beginning + middle + end

3B42_Daily.19980101.7.nc4
365


### Making a 3D array of all the rainfall values of the entire grid for a given year

In [3]:
def MultipleCellcal(year):
    beginning = "TRMM data/3B42_Daily."
    end = ".7.nc4"
    Loop = YearLoop(year,year)
    temp = np.zeros([1440,400])
    NumberOfDays = len(Loop)
    for i in range(len(Loop)):
        middle = Loop[i]
        filename = beginning + middle + end
        ds = xr.load_dataset(filename)
        precp = ds.precipitation
        temp = np.dstack((temp, precp))
    temp = np.nan_to_num(temp)  # To replace nan with 0 and make sure that the sort here after puts these "zeros" at the bottom of the array
    temp = np.sort(temp)
    Top25 = temp[:,:,-25:]
    np.save("MultiplecellTop25"+str(year),Top25)

#### Running the code above for the entire data set

In [4]:
for i in np.arange(1998, 2019+1):
    MultipleCellcal(i)

### Looking at a bloack of 5 years to determine the top 25 for every single cell

In [17]:
beginning = "MultiplecellTop25"
end = ".npy"
    
for k in np.arange(17+1):
    year = 1998 + k
    temp =[]
    for i in np.arange(year, year+4+1):
        middle = i
        name = beginning + str(middle) + end
        if i == year:
            temp = np.load(name)
        else:
            temp = np.dstack((temp, np.load(name)))
    temp = np.sort(temp)
    Top25 = temp[:,:,-25:]
    np.save("Multiple5yearTop" + str(year), Top25)
    year5 = Top25[:,:,24]
    year1 = Top25[:,:,-5]
    year02 = Top25[:,:,0]
    np.save("Multipleyear5max" + str(year) ,year5)
    np.save("Multipleyear1max" + str(year) ,year1)
    np.save("Multipleyear0,2max" + str(year) ,year02)

### Fitting a line to the rainfall for 5, 1 and 0.2 year repeat times just calculated for every single cell

In [20]:
def func(x, a, b):
    y = a*x + b
    return y

alphaArray02 = []
alphaArray1 = []
alphaArray5 = []

betaArray02 = []
betaArray1 = []
betaArray5 = []

gammaArray02 = []
gammaArray1 = []
gammaArray5 = []
# Loops through 0.2, 1 and 5 year return times
for n in np.arange(3):
    num = ['0,2', '1', '5']
    
    alpha = np.zeros([1440,400])
    beta = np.zeros([1440,400])
    gamma = np.zeros([1440,400])
    y = []
    
    # Loops through the files for every bloack of 5 years and stacks them
    for i in np.arange(18):
        name = "Multipleyear" + str(num[n]) + "max" + str(1998 + i) + ".npy"
        data = np.load(name)
        if i == 0:
            y = data
        else:
            y = np.dstack((y, data))
    
    x = np.arange(2000, 2000+18)
    NumberofNAN = 0
    
    # Loops through every cell and calculates the fit line
    for i in range(1440):
        for j in range(400):
            if np.any(np.isnan(y[i,j])):
                alpha[i,j] = 0
                NumberofNAN += 1
            else:
                a, b = optimize.curve_fit(func, xdata = x, ydata = y[i,j])[0]
                alpha[i,j] = a
                beta[i,j] = b
                gamma[i,j] = func(2000, a, b)
                
    if n == 0:
        alphaArray02 = alpha
        betaArray02 = beta
        gammaArray02 = gamma
    elif n == 1:
        alphaArray1 = alpha
        betaArray1 = beta
        gammaArray1 = gamma
    elif n == 2:
        alphaArray5 = alpha
        betaArray5 = beta
        gammaArray5 = gamma

C:\Users\woute\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


### Saving desired data as netcdf4 files (manipulation required).

In [39]:
longitude = np.linspace(-180, 180, 1440)
latitude = np.linspace(-50, 50, 400)


### only make one netcdf4 file at a time ###
# For calculating the percentage increase difference between two recurance periods and making a netcdf4 file 
#df = xr.DataArray((np.divide(alphaArray1, gammaArray1)-np.divide(alphaArray02, gammaArray02)), coords=[('lon', longitude), ('lat', latitude)])

# Making a netcdf4 file for the increase in precipitaion in mm/day/year
#df = xr.DataArray((alphaArray5), coords=[('lon', longitude), ('lat', latitude)])

### Saving the newly made netcdf4 file under an appropriate name ###
#df.to_netcdf('netcdfperc1minus02year_withnanto0.nc')